In [1]:
# Started 8 June 2016
# Based on learning unscented kalman filter
# Import the required libraries 
import numpy as np
#from numpy import linalg
import scipy.linalg
#from numpy.random import randn
import matplotlib.pyplot as plt

In [2]:
# Validated 8 June 2016
def fnCreateConcatenatedRmat(R,Rinv,stacklen):
    L = [R]; Linv = [Rinv];
    for index in range (0,stacklen):
        L.append(R);
        Linv.append(Rinv);
    ryn = scipy.linalg.block_diag(*L);
    ryninv = scipy.linalg.block_diag(*Linv);
    return ryn,ryninv;

In [3]:
def fnH(X):
    Y = np.zeros([2],dtype=float);
    Y[0] = np.linalg.norm(X);
    Y[1] = np.arctan2(X[1],X[0]);
    return Y

In [4]:
def fnUT_sigmas(X,P,params_vec):
    # Implementation of ut_sigmas.m of the ekfukf toolbox
    A = np.linalg.cholesky(P);
    n = params_vec[3]; kappa = params_vec[2];
    sigmas = np.vstack((np.zeros_like(X),A ,-A) );
    c  = n + kappa;
    sigmas = np.sqrt(c)*sigmas;
    sigmas  = np.add(sigmas,np.tile(X,(2*n+1,1)));
    return sigmas

In [5]:
alpha =1;
beta =2;
kappa = 5;
n =2;
params_vec =np.array([alpha,beta,kappa,n],dtype=int);
M = np.array([1.,2.]); P = np.diag([4.,5.]);

sigmax = fnUT_sigmas(M ,P,params_vec)
print sigmax
# Validated: matches with the results from ut_sigmas.m of the ekfukf toolbox.

[[ 1.          2.        ]
 [ 6.29150262  2.        ]
 [ 1.          7.91607978]
 [-4.29150262  2.        ]
 [ 1.         -3.91607978]]


In [6]:
def fnUT_weights(params):
    # X - state vector
    # P - covariance matrix
    # params_vec = [alpha,beta,kappa,n]
    # emulates ut_weights.m of the ekfukftoolbox
    alpha = float(params[0]); 
    beta = float(params[1]);
    kappa = float(params[2]); 
    n = params[3];
    lambd = np.square(alpha) * (float(n) + kappa) - float(n);
    Weights_Mean = np.zeros((2*n+1),dtype=float);
    Weights_Cov = np.zeros((2*n+1),dtype=float);
    Weights_Mean[0] = lambd/(float(n)+lambd);
    Weights_Cov[0] = lambd/(float(n)+lambd) + (1-np.square(alpha) + beta);
    for index in range(1,2*n+1):
        Weights_Mean[index] = 1 / (2 * (float(n) + lambd));
        Weights_Cov[index] = Weights_Mean[index];
    return Weights_Mean,Weights_Cov

In [7]:
Weights_Mean, Weights_Cov =  fnUT_weights(params_vec)
print Weights_Mean
print Weights_Cov
# Validated fnUT_weights

[ 0.71428571  0.07142857  0.07142857  0.07142857  0.07142857]
[ 2.71428571  0.07142857  0.07142857  0.07142857  0.07142857]


In [10]:
def fnUT_transform(M,P,fnG,params):
    # emulates ut_transform.m of the ekfukftoolbox.
    # M - mean state vector
    # P - corresponding covariance matrix
    # fnG - nonlinear measurement function
    # params - vector of parameters for the unscented transform.
    
    # Form the sigma points of x
    sigmas = fnUT_sigmas(M,P,params);
    # Compute weights
    Wm,Wc = fnUT_weights(params);  
    
    n = params[3];
    # Propagate sigma points through the (non)linear model.
    yo = fnG(sigmas[0,:]);
    
    Y = np.zeros([np.shape(yo)[0],2*n+1],dtype=float); # sigma points of y
    Y[:,0] = yo;
    mu = np.dot(Wm[0],Y[:,0]);
    for index in range(1,2*n+1):
        Y[:,index] = fnG(sigmas[index,:]);
        mu += np.dot(Wm[index],Y[:,index]);

    Sk  = np.zeros([np.shape(Y)[0],np.shape(Y)[0]],dtype=float);
    Ck  = np.zeros([np.shape(Y)[0],np.shape(Y)[0]],dtype=float);
    
    for index in range (0,2*n+1):
        diff = np.subtract(Y[:,index],mu);
        produ = np.multiply.outer(diff,diff);
        Sk = np.add(Sk,np.dot(Wc[index],produ)); 
        diff1 = np.subtract(sigmas[index,:] ,M);
        produ1 = np.multiply.outer(diff1,diff);
        Ck = np.add(Ck,np.dot(Wc[index],produ1));    
    return sigmas,Wm,Wc,Y,mu,Sk,Ck

In [11]:
M = np.array([3,2],dtype=float) 
[sigmas,Wm,Wc,Y,mu,Sk,Ck] = fnUT_transform(M,P,fnH,params_vec);
print sigmas

[[ 3.          2.        ]
 [ 8.29150262  2.        ]
 [ 3.          7.91607978]
 [-2.29150262  2.        ]
 [ 3.         -3.91607978]]


In [12]:
print Y

[[ 3.60555128  8.52930336  8.46547808  3.04154307  4.9331208 ]
 [ 0.5880026   0.23668952  1.20854488  2.42401585 -0.91708809]]


In [13]:
print mu
print Sk
print Ck

[ 4.35892558  0.63087059]
[[ 4.13491349 -0.0925349 ]
 [-0.0925349   0.44074739]]
[[ 2.07417842 -0.82673164]
 [ 1.49269339  0.89824387]]


In [ ]:
# Validated 8 June 2016
# Results match with testfunctionsforpython.m used to verify the ekfukf toolbox